In [ ]:
import numpy as np
import pandas as pd

## Dataset
The two datasets used in this project are the tmdb 5000 movies and the tmdb 5000 credits. 
These datasets contain movie metadata such as titles, genres, keywords, cast, and overviews. 

This data is used to construct feature vectors for each movie, which 
are then compared using cosine similarity to identify similar movies.

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv.zip')
credits = pd.read_csv('tmdb_5000_credits.csv.zip')

In [ ]:
movies.head()

In [ ]:
credits.head(1)

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.info()

## Data Cleaning

In [ ]:
#choosing the columns that matter
movies = movies[['movie_id','genres','keywords','title','overview','cast','crew']]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True) #clearing missing data

In [ ]:
movies.duplicated().sum() #no duplicate data in this dataset

In [ ]:
movies.iloc[0].genres

In [ ]:
#converting the list of dictionaries to a plain python list
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
import ast

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
#creating a list of only "character" values(names) for the cast column
def convert2(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l

In [ ]:
movies['cast']=movies['cast'].apply(convert2)

In [ ]:
movies.head()

In [ ]:
#fetching only directors from the crew column
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
 movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
#removing whitespaces
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
#converting to lowercase
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df.head()

In [ ]:
import nltk #Natural Language Toolkit

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
#reducing each word to its root form
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

## Vectorisation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
new_df["tags"] = new_df["tags"].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
#final function
def recommend(movie):
    movie_index = new_df[new_df["title"]==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

## Example Recommendations

In [102]:
recommend("Avatar")

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [104]:
recommend("Titanic")

Raise the Titanic
Captain Phillips
The Notebook
In the Heart of the Sea
Ghost Ship
